In [1]:
# Imports
import pickle, json
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [2]:
# Importing, opening pickled array, size brought down from 20.4 GBs to .5 GBs pre-pickling
with open('PreProcessing/kickXY.pkl', 'rb') as fp:
	base = pickle.load(fp)

In [3]:
# Key for array column names
# key = [0'unid', 1'timeline', 2'goalUSD', 3'category', 4'text', 5'state']

# Turning array input from scraped data into dataframe
bigDF = pd.DataFrame(data=base)

# Splitting category 'slug' into category & subcategory
bigDF[3] = bigDF[3].str.split('/', n=1)
bigDF[6] = [bigDF[3][i][-1] for i in range(len(bigDF))]
bigDF[3] = [bigDF[3][i][0] for i in range(len(bigDF))]

# Dropping 'live', 'canceled', 'suspended' records and removing dupes (3.6M->221K)
states = ['live', 'canceled', 'suspended']
for i in range(len(states)):
    bigDF.drop(bigDF[bigDF[5] == states[i]].index, inplace=True)

bigDFunique = bigDF.drop_duplicates(keep='last')

# Dropping unique_id column post dupe-removal
bigDFunique.drop(columns=0, inplace=True)

/Users/rowenwitt/.local/share/virtualenvs/Backup-YjIEX9Sx/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [4]:
# Saving cleaned, de-duped, state-simplified csv (3.6M -> 221K)
bigDFunique.to_csv('cleanKick.csv')

In [65]:
# Normalize funding goal

bigDFunique[2] = bigDFunique[2] / bigDFunique[2].max()

<ipython-input-65-724191fddc98>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigDFunique[2] = bigDFunique[2] / bigDFunique[2].max()


In [67]:
# Label encode category, state, subCategory
LeCat = LabelEncoder()
LeSubCat = LabelEncoder()
LeState = LabelEncoder()

# Fit encoders on respective columns
LeCat.fit(bigDFunique[3])
LeSubCat.fit(bigDFunique[6])
LeState.fit(bigDFunique[5])

# Apply encodings to columns
bigDFunique[3] = LeCat.transform(bigDFunique[3])
bigDFunique[6] = LeSubCat.transform(bigDFunique[6])
bigDFunique[5] = LeState.transform(bigDFunique[5])

<ipython-input-67-e6fc0a584793>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigDFunique[3] = LeCat.transform(bigDFunique[3])
<ipython-input-67-e6fc0a584793>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigDFunique[6] = LeSubCat.transform(bigDFunique[6])
<ipython-input-67-e6fc0a584793>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [68]:
# create dict maps
def ones(imp):
    out = []
    for i in range(len(imp)):
        out.append(i)
    
    dmap = dict(zip(out, imp))
    return dmap

StateMap = ones(LeState.classes_)
CatMap = ones(LeCat.classes_)
SubCatMap = ones(LeSubCat.classes_)

In [8]:
# SubCatMap

In [69]:
# Instantiating Tokenizer
tok = Tokenizer()

# Fitting tokenizer on df text array
tok.fit_on_texts(texts=bigDFunique[4])

# Getting word index from tokenizer
dex = tok.word_index

# Keep work index to 1000 words
tok.num_words = 1000

# Creating sequences from df text array
seqs = tok.texts_to_sequences(bigDFunique[4])
max_features = len(dex.values()) + 1
maxlen = 43

# Saving tokenizer to pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Padding sequence length
bigX = sequence.pad_sequences(seqs, maxlen)

# Creating list of reconstructed text
recons = tok.sequences_to_texts(seqs)

In [70]:
bigX[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [71]:
bigDFunique.shape

(211038, 49)

In [72]:
# Giving text encodings (bigX) own column 7-49
for j in range(7, (50)):
    f = j-7
    bigDFunique[j] = [bigX[i][f] for i in range(len(bigDFunique))]

<ipython-input-72-b0ee71e790d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigDFunique[j] = [bigX[i][f] for i in range(len(bigDFunique))]


In [73]:
# dropping out column 4 (unencoded text), re-naming columns, renaming df to df
df = bigDFunique.drop(columns=[4])

newColumns = [i for i in range(48)]
df.columns = newColumns

# Splitting into X and Y (4 = success/failure)
target = 3
X = df.drop(columns=target)
Y = df[target]

In [74]:
df

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
21294,30,0.000052,12,1,82,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21976,24,0.000026,1,1,66,0,0,0,0,0,...,199,24,51,536,3,3,27,85,330,496
29021,60,0.000072,1,0,66,0,0,0,0,0,...,22,650,198,630,1,370,63,3,107,2
44904,31,0.000010,1,1,6,0,0,0,0,0,...,146,237,5,2,266,76,113,635,620,2
45339,31,0.000037,8,1,106,0,0,0,0,0,...,0,0,2,383,36,262,171,133,814,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658378,17,0.000004,0,0,28,0,0,0,0,0,...,0,119,22,54,50,531,27,6,2,408
3658379,30,0.000066,12,0,159,0,0,0,0,0,...,4,534,61,6,239,35,12,42,28,759
3658380,20,0.000020,0,1,70,0,0,0,0,0,...,634,43,745,5,2,240,2,910,3,2
3658381,30,0.000033,12,0,15,0,0,0,0,0,...,41,648,6,2,5,3,2,3,291,5


In [75]:
X

,0,1,2,4,5,6,7,8,9,10,...,38,39,40,41,42,43,44,45,46,47
21294,30,0.000052,12,82,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21976,24,0.000026,1,66,0,0,0,0,0,0,...,199,24,51,536,3,3,27,85,330,496
29021,60,0.000072,1,66,0,0,0,0,0,0,...,22,650,198,630,1,370,63,3,107,2
44904,31,0.000010,1,6,0,0,0,0,0,0,...,146,237,5,2,266,76,113,635,620,2
45339,31,0.000037,8,106,0,0,0,0,0,0,...,0,0,2,383,36,262,171,133,814,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658378,17,0.000004,0,28,0,0,0,0,0,0,...,0,119,22,54,50,531,27,6,2,408
3658379,30,0.000066,12,159,0,0,0,0,0,0,...,4,534,61,6,239,35,12,42,28,759
3658380,20,0.000020,0,70,0,0,0,0,0,0,...,634,43,745,5,2,240,2,910,3,2
3658381,30,0.000033,12,15,0,0,0,0,0,0,...,41,648,6,2,5,3,2,3,291,5


In [76]:
# Creating X,Y Train and X,Y Validation sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
((X_train.shape, y_train.shape),(X_test.shape, y_test.shape))

(((147726, 47), (147726,)), ((63312, 47), (63312,)))

In [77]:
# Convert X_train to numpy array
arr = X_train.to_numpy()
# Reshape numpy array into 3D tensor
tarr = arr.reshape(147726, 47, 1)
# Convert ints to floats
X_train = np.asarray(tarr).astype('float32')
y_train = np.asarray(y_train).astype('float32')

# Convert X_test to numpy array
arr = X_test.to_numpy()
# Reshape numpy array into 3D tensor
varr = arr.reshape(63312, 47, 1)
# Convert ints to floats
X_test = np.asarray(varr).astype('float32')
y_test = np.asarray(y_test).astype('float32')

In [78]:
((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

(((147726, 47, 1), (147726,)), ((63312, 47, 1), (63312,)))

In [94]:
# Separate words from columns [0:3] [0, 1, 2, 4] from columns [4:] for separate models
print(len(X.iloc[0][:4]), len(X.iloc[0][4:]))
# X_train_words # = [X_train[i]]

4 43


In [35]:
len(dex)

131099

In [96]:
print('building model')
maxlen = 43
dim = len(dex)
model = Sequential()

""" Changes made
- Try dropping out unid > Holy shit it broke the model, was it keying on unid?
    Nope, I'm an idiot and it frameshifted one to left (durrrr)
- Added second LSTM layer, has not really impacted accuracy
- Added dense layer
- Normalized usd_goal input
- Set up to do feature union of better NLP words & normal cols 0-3
"""

# Try embeddings (will it work if only 43/49 columns are encoded words?)

# Try normalizing inputs (just words? everything individually?)

# Try playing with layers (Add'l dense layer?  Add'l LSTM layer?)

# model.add(Embedding(input_dim=15000,
# 					embeddings_initializer="uniform", output_dim=256))

model.add(Embedding(500000, 47))

model.add(LSTM(128, dropout=0.1, return_sequences=True))

#model.add(LSTM(64, input_shape=(47, 1), return_sequences=True))

#model.add(LSTM(32, input_shape=(47, 1), return_sequences=False))

#model.add(Dense(32, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',
			  metrics=['accuracy'])



building model


In [31]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          12800000  
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 128)         98816     
_________________________________________________________________
dense_4 (Dense)              (None, None, 1)           129       
Total params: 12,898,945
Trainable params: 12,898,945
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Early stopping on val_loss
stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)

print("training")
history = model.fit(tarr, y_train,
				   validation_data=(varr, y_test),
				   batch_size=1024,
				   epochs=2000,
				   verbose=1,
                   callbacks=[stop],
                   workers=10)

print(history)
print('---')

training
Epoch 1/2000
145/145 [==============================] - 107s 727ms/step - loss: 0.5935 - accuracy: 0.6693 - val_loss: 0.5169 - val_accuracy: 0.7318
Epoch 2/2000
145/145 [==============================] - 87s 603ms/step - loss: 0.4989 - accuracy: 0.7345 - val_loss: 0.4867 - val_accuracy: 0.7363
Epoch 3/2000
145/145 [==============================] - 97s 670ms/step - loss: 0.4790 - accuracy: 0.7396 - val_loss: 0.4709 - val_accuracy: 0.7407
Epoch 4/2000
145/145 [==============================] - 94s 646ms/step - loss: 0.4621 - accuracy: 0.7465 - val_loss: 0.4590 - val_accuracy: 0.7490
Epoch 5/2000
145/145 [==============================] - 95s 657ms/step - loss: 0.4565 - accuracy: 0.7502 - val_loss: 0.4573 - val_accuracy: 0.7513
Epoch 6/2000
128/145 [=========================>....] - ETA: 10s - loss: 0.4517 - accuracy: 0.7545

In [ ]:
import shap

# load your data here, e.g. X and y
# create and fit your model here

# load JS visualization code to notebook
shap.initjs()

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

shap.summary_plot(shap_values, X, plot_type="bar")

In [ ]:
# Save first model (get a point on the board!)
model.save('models/model2')
print('saved successfully')

In [ ]:
import tensorflow as tf
# Convert model to tflite model (it can't be that simple)
converter = tf.lite.TFLiteConverter.from_saved_model('models')
tflite_model = converter.convert()

# Resave model
with open('model1.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Will take forever to run, accuracy is so-so for 3-category class
# Options are
#    1. Sample data set, 3,600,000 is too many (weak)
#    2. Set up EC2 instance, pay like 10 dollars/hour (cool, difficult)
#    3. Increase batch-size to the 10k range (dumb?)
#    4. Decrease validation size to 1/5
#                Test set size = 1,830,912 ... 1/5 = 366,182.4
# Blurb dataset is 150Char, Descrip dataset is 3,500Char
# Will this impact training time?  Maybe not?
